In [25]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import cv2
from tqdm.auto import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
import pandas as pd
import tensorflow as tf
import speech_recognition as sr
from gtts import gTTS
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
data_dir=r'C:\Users\hp\Desktop\SIH\skin_disease_sih\training\train_set'
# data = pd.read_csv(data_dir)

In [3]:
img_height,img_width=224,224
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 824 files belonging to 7 classes.


Using 660 files for training.


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
img_height,img_width=224,224
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 824 files belonging to 7 classes.
Using 164 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['BA- cellulitis', 'BA-impetigo', 'FU-athlete-foot', 'FU-nail-fungus', 'FU-ringworm', 'VI-chickenpox', 'VI-shingles']


In [7]:
from keras.callbacks import ReduceLROnPlateau
learning_control = ReduceLROnPlateau(monitor = 'val_acc' , patience=3 , verbose = 1 , factor = 0.5 , lr = 0.0001)

In [8]:
# resnet_model = Sequential()

# pretrained_model= tf.keras.applications.ResNet50(include_top=False,
#                    input_shape=(180,180,3),
#                    pooling='avg',classes=7,
#                    weights='imagenet')
# for layer in pretrained_model.layers:
#         layer.trainable=False

# resnet_model.add(pretrained_model)
# resnet_model.add(Flatten())
# resnet_model.add(Dense(128, activation='relu'))
# resnet_model.add(Dense(64,activation='relu'))
# resnet_model.add(Dense(3,activation='softmax'))

resnet_model = Sequential()

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(224, 224, 3),
    pooling='avg',
    classes=7,  # Ensure this matches the number of classes in your task
    weights='imagenet'
)

for layer in pretrained_model.layers:
    layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(128, activation='relu'))
resnet_model.add(Dense(64, activation='relu'))
resnet_model.add(Dense(7, activation='softmax'))  # Change the units to match the number of classes




In [9]:

sgd = SGD(lr = 0.01 , clipvalue = 0.5)
resnet_model.compile(optimizer = sgd , loss="categorical_crossentropy" , metrics =["accuracy"])

In [10]:
# resnet_model.compile(optimizer='Adam'(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# from tensorflow.keras.optimizers import Adam

# # Define the optimizer with the learning rate
# optimizer = Adam(learning_rate=0.001)

# # Compile the model using the defined optimizer
# resnet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

from tensorflow.keras.optimizers import Adam

# Define the optimizer with the learning rate
optimizer = Adam(learning_rate=0.001)

# Compile the model using the defined optimizer
resnet_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [11]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper (ModuleWrap  (None, 2048)              0         
 per)                                                            
                                                                 
 module_wrapper_1 (ModuleWr  (None, 128)               262272    
 apper)                                                          
                                                                 
 module_wrapper_2 (ModuleWr  (None, 64)                8256      
 apper)                                                          
                                                                 
 module_wrapper_3 (ModuleWr  (None, 7)                 455       
 apper)                                                 

Total params: 23858695 (91.01 MB)
Trainable params: 270983 (1.03 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [26]:
# Define the EarlyStopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [29]:
epochs = 10
history = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    batch_size = 32,
    epochs=epochs,
    callbacks = [early_stop]
)

Epoch 1/10
21/21 [==============================] - 51s 2s/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0944 - val_accuracy: 0.9756
Epoch 2/10
21/21 [==============================] - 42s 2s/step - loss: 8.7873e-04 - accuracy: 1.0000 - val_loss: 0.0938 - val_accuracy: 0.9756
Epoch 3/10
21/21 [==============================] - 46s 2s/step - loss: 7.8531e-04 - accuracy: 1.0000 - val_loss: 0.0946 - val_accuracy: 0.9756
Epoch 4/10
21/21 [==============================] - 47s 2s/step - loss: 6.8963e-04 - accuracy: 1.0000 - val_loss: 0.0956 - val_accuracy: 0.9756
Epoch 5/10
21/21 [==============================] - 43s 2s/step - loss: 6.1368e-04 - accuracy: 1.0000 - val_loss: 0.0972 - val_accuracy: 0.9756


In [ ]:
from flask import Flask, jsonify, request
import numpy as np
import tensorflow as tf

app = Flask(__name__)

# Load your pre-trained model
model = tf.keras.models.load_model('modelsih.h5')

# Example endpoint for prediction
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()  # Get data from POST request
    # Process the data as needed (e.g., convert to numpy array)
    input_data = np.array(data['input'])
    
    # Make predictions using the loaded model
    # predictions = model.predict(input_data)
    
    # Return predictions as JSON
    return jsonify({'predictions': resnet_model.predict(img_array).tolist()})  # Convert predictions to JSON

if __name__ == '__main__':
    app.run(debug=True)  # Run the Flask app


In [30]:
from tensorflow.keras.models import load_model, save_model

# Assuming 'model' is your pretrained model
resnet_model.save('pretrained_model.h5')
# resnet_model.save_weights('pretrained_weights.h5')

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NotImplementedError: 
Layer ModuleWrapper was created by passing
non-serializable argument values in `__init__()`,
and therefore the layer must override `get_config()` in
order to be serializable. Please implement `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2, **kwargs):
        super().__init__(**kwargs)
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config

In [ ]:
# import h5py

# # Open the HDF5 file
# with h5py.File('pretrained_model.h5', 'r') as file:
#     # List all groups and datasets within the file
#     def print_attrs(name, obj):
#         print(name)
#         for key, val in obj.attrs.items():
#             print(f"    {key}: {val}")

#     file.visititems(print_attrs)

In [32]:

def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("output1.mp3")
    os.system("mpg123 output1.mp3")  # or use any other audio player suitable for your system
speak("HELLO SIR , PLEASE CLICK THE BLUE BUTTON TO CLICK AN IMAGE OR RED BUTTON TO TAKE A  VIDEO")
img_path = r'C:\Users\hp\Desktop\SIH\skin_disease_sih\training\train_set\test_image.jpeg'  # Replace with your image file path

def speak(text):
    tts = gTTS(text=text, lang='hi')
    tts.save("output2.mp3")
    os.system("mpg123 output2.mp3") 
speak("thodi der wait kijiye")
img = image.load_img(img_path, target_size=(224, 224))  # Adjust target_size as needed for your model
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

In [17]:
arr = resnet_model.predict(img_array)
print(arr)
index_of_largest = np.argmax(arr)
arr[0][index_of_largest]

1/1 [==============================] - 4s 4s/step
[[9.4792180e-05 1.1248579e-03 1.9411850e-07 4.6054702e-05 9.9868172e-01
  8.0003062e-08 5.2347259e-05]]


0.9986817

In [18]:
class_labels = ['Cellulitis', 'Impetigo', 'Athlete Foot', 'Nail Fungus', 'Ringworm' , 'Chickenpox', 'Shingles' ]
print("The probability of the disease",class_labels[index_of_largest] ," is " , arr[0][index_of_largest])
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("output2.mp3")
    os.system("mpg123 output2.mp3") 
speak("PLEASE WAIT FOR A WHILE")

The probability of the disease Ringworm  is  0.9986817


In [19]:
import cv2
image=cv2.imread(r'C:\Users\hp\Desktop\SIH\skin_disease_sih\training\train_set\test_image.jpeg')
image_resized= cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)
pred=resnet_model.predict(image)
print(pred)
output_class=class_names[np.argmax(pred)]
print("The predicted class is", output_class)

(1, 224, 224, 3)
1/1 [==============================] - 2s 2s/step
[[3.5384217e-06 7.8786897e-08 4.6282933e-09 9.4208553e-06 9.9988794e-01
  2.8019222e-09 9.8895529e-05]]
The predicted class is FU-ringworm


In [ ]:
# C:\Users\hp\Desktop\SIH\skin_disease_sih\training

In [21]:
# import speech_recognition as sr
# from gtts import gTTS
# import os

# Function to recognize speech
def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    
    try:
        text = recognizer.recognize_google(audio)
        return text.lower()  # Return the recognized text in lowercase
    except sr.UnknownValueError:
        return "Sorry, I didn't understand that."
    except sr.RequestError:
        return "Sorry, there was an error recognizing the speech."

# Function to speak
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("output.mp3")
    os.system("mpg123 output.mp3")  # or use any other audio player suitable for your system
speak("The probability of the disease Ringworm  is  99.99918%")
# Your ML model to process the recognized text (replace this with your model integration)
def process_command(command):
    # Your model logic goes here
    # Example:
    if "hello" in command:
        return "Hi there!"
    else:
        return "Command not recognized."

# Main loop
while True:
    user_input = listen()  # Get user's voice command
    print("User said:", user_input)
    
    response = process_command(user_input)  # Process the command using your ML model
    
    speak(response)  # Speak the response


Listening...
User said: Sorry, I didn't understand that.
Listening...
User said: hello there mentor is a bitch their mentor is a bitch their mentor is a which there mentor is a bitch hey hello hello i don't understand
Listening...


KeyboardInterrupt: 

AttributeError: 'Sequential' object has no attribute 'h5'

In [ ]:

model = resnet_model  # Load your trained Keras model
image_height = 224

# Define ImageDataGenerator for testing data
test_datagen = ImageDataGenerator(rescale=1. / 255)  # Normalize pixel values
batch_size = 32
# Load testing data from directory
test_generator = test_datagen.flow_from_directory(
    r'C:\Users\hp\Desktop\SIH\skin_disease_sih\training\test_set',
    target_size=(image_height, image_height),  # Set image size as required
    batch_size=batch_size,
    class_mode='categorical',  # Adjust based on your problem (e.g., 'binary', 'categorical')
    shuffle=False  # Ensure the order is maintained for calculating confidence scores
)

# Predict on testing data and get confidence scores
confidence_scores = model.predict_generator(test_generator)
print(confidence_scores)


Found 208 images belonging to 7 classes.


C:\Users\hp\AppData\Local\Temp\ipykernel_25736\940447153.py:17: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  confidence_scores = model.predict_generator(test_generator)


[[1.5235197e-02 1.4080690e-03 3.6288972e-03 ... 4.1197619e-01
  5.3042453e-04 6.3429889e-03]
 [1.4146981e-02 1.4605016e-03 3.3801973e-03 ... 4.2742375e-01
  5.3060672e-04 6.1629605e-03]
 [1.2745012e-02 1.1785305e-03 2.7713107e-03 ... 4.3474007e-01
  4.1956609e-04 5.1961853e-03]
 ...
 [1.5081271e-02 1.6227755e-03 3.6844886e-03 ... 4.2829046e-01
  5.8251451e-04 6.5011661e-03]
 [1.5050463e-02 1.6006541e-03 3.6874877e-03 ... 4.3010622e-01
  5.6558434e-04 6.3350247e-03]
 [1.4282130e-02 1.5584549e-03 3.5180172e-03 ... 4.3903843e-01
  5.3380587e-04 5.8330777e-03]]


In [ ]:
# from sklearn.metrics import accuracy_score

# # Assuming predicted_labels and true_labels are available
# accuracy = accuracy_score(true_labels, predicted_labels)
# print(f"Accuracy: {accuracy * 100:.2f}%")

NameError: name 'true_labels' is not defined

In [ ]:
# model.save(resnet_model.hdf5)

AttributeError: 'Sequential' object has no attribute 'hdf5'